In [5]:
import damask
import numpy as np

## Voronoi geometry with 4096 cells and 50 grains with initial temperature as 303 K

In [6]:
cells = [16,16,16]
size = np.array(cells)*1.0
N_grains = 50
seeds = damask.seeds.from_random(size,N_grains,cells)
grid = damask.GeomGrid.from_Voronoi_tessellation(cells,size,seeds)

In [7]:
grid.initial_conditions["T"] = 303.0

In [8]:
grid.save("thermal/Voronoi.vti")

## Load file which specifies the thermal solver

In [14]:
load = damask.YAML.load("reference_input/tensionX.yaml")

In [15]:
load

solver: {mechanical: spectral_basic}
loadstep:
- boundary_conditions:
    mechanical:
      dot_F:
      - [0.001, 0, 0]
      - [0, x, 0]
      - [0, 0, x]
      P:
      - [x, x, x]
      - [x, 0, x]
      - [x, x, 0]
  discretization: {t: 10.0, N: 100}
  f_out: 4

In [16]:
load["solver"]["thermal"] = "spectral"

In [17]:
load.save("thermal/tensionX.yaml")

In [38]:
m = damask.YAML.load("thermal/material.yaml")

## Run mechanics at a different temperature (no thermal solving)

In [39]:
m_new = m

In [40]:
m_new["phase"]["IFSteel"]["thermal"]

{'references': ['J.G. Hust and A.B. Lankford, Thermal Conductivity of Aluminum, Copper, Iron, and Tungsten from 1K to the Melting Point, US Department of Commerce, Boulder, Colorado, 1984, fit to Tab. 4.4.1 (RRR=300, T_min=150K, T_max=1000K)',
  'https://www.engineeringtoolbox.com/specific-heat-metals-d_152.html'],
 'output': ['T'],
 'K_11': 80.34,
 'K_11,T': -0.1216,
 'K_11,T^2': 0.0001327,
 'K_11,T^3': -7.993e-08,
 'T_ref': 293.15,
 'C_p': 450.0,
 'source': [{'type': 'dissipation', 'kappa': 0.9}]}

In [41]:
m_new["phase"]["IFSteel"]["thermal"]["K_11"] = 0.0
m_new["phase"]["IFSteel"]["thermal"]["K_11,T"] = 0.0
m_new["phase"]["IFSteel"]["thermal"]["K_11,T^2"] = 0.0
m_new["phase"]["IFSteel"]["thermal"]["K_11,T^3"] = 0.0

In [42]:
m_new["phase"]["IFSteel"]["thermal"].pop("source", None)

[{'type': 'dissipation', 'kappa': 0.9}]

In [43]:
m_new.save("thermal/material_no_temp_solve.yaml")

## thermo-mechanics with conduction

In [44]:
m = damask.YAML.load("thermal/material.yaml")

In [45]:
m.save("thermal/material_conduction.yaml")

In [46]:
m = damask.YAML.load("thermal/material.yaml")

In [47]:
m_new["phase"]["IFSteel"]["thermal"]["K_11"] = 0.0
m_new["phase"]["IFSteel"]["thermal"]["K_11,T"] = 0.0
m_new["phase"]["IFSteel"]["thermal"]["K_11,T^2"] = 0.0
m_new["phase"]["IFSteel"]["thermal"]["K_11,T^3"] = 0.0

In [48]:
m.save("thermal/material_adiabatic.yaml")